# Robo-MZOIP - ocjene prihvatljivosti za ekološku mrežu Natura 2000

## Uvod

### Natura 2000

Zakonski okvir: EU Direktiva o pticama (79/409/EEC), Direktiva o staništima (92/43/EEC)
http://ec.europa.eu/environment/nature/legislation/birdsdirective/index_en.htm
http://ec.europa.eu/environment/nature/legislation/habitatsdirective/index_en.htm

 * Područja očuvanja značajna za ptice
 * Područja očuvanja značajna za vrste i staništa

 * Postupak ocjene prihvatljivosti za ekološku mrežu
  - Prethodna ocjena
  - Glavna ocjena

## Problem

 * Nepregledna stranica Ministarstva zaštite okoliša i energetike.
 * Dokumenti se dodaju na način da je nemoguće sustavno pratiti njihovo dodavanje.

## Rješenje

 * Web scraper (python3 + requests + beautiful soup 4)
 * Twitter

In [82]:
import argparse
from datetime import datetime
import os
import re
import sys
import requests
from bs4 import BeautifulSoup
import pandas as pd

Definicija stranice koja nas zanima:

In [83]:
BASE_URL_OPEM = 'http://mzoe.hr/hr/priroda/ekoloska-mreza-natura-2000.html'

Dohvaćanje html stranice, i kreiranje `soup` objekta sa stranice Ministarstva:

In [84]:
def get_soup(full_url):
    """Dohvaca soup objekt s mreznih stranica.

    Args:
        full_url (str): path do stranice.
    Returns:
        soup: beautifulsoup object.
    """
    r = requests.get(full_url)
    soup = BeautifulSoup(r.content, 'lxml')
    return soup

In [85]:
soup = get_soup(BASE_URL_OPEM)

### Glavne ocjene u nadležnosti Ministarstva

In [86]:
opem = soup.find('div', 'accordion').find_all('div', 'accordion')
a = opem[0].find_all('a')

In [87]:
opem_raw = []
for link in a:
    red = [link.parent.parent.parent.previous_sibling.previous_sibling.get_text().strip(),
           link.get_text(),
           link.get('href')]
    opem_raw.append(red)
opem_pd = pd.DataFrame(opem_raw)

In [88]:
opem_pd.head(10)

,0,1,2
0,Uređenje vodnog puta rijeke Dunav kod Sotina n...,16.02.2018. - INFORMACIJA o zahtjevu za provođ...,/doc/16022018_-_informacija_o_zahtjevu_za_prov...
1,Uređenje vodnog puta rijeke Dunav kod Sotina n...,24.11.2015. - RJEŠENJE Ministarstva od 11. stu...,/doc/24112015_-_rjesenje_ministarstva_o_odbija...
2,Uređenje vodnog puta rijeke Dunav kod Sotina n...,02.10.2015. - INFORMACIJA o zahtjevu za provođ...,/doc/02102015_-_informacija_o_zahtjevu_za_prov...
3,Izgradnja cestovnog mosta preko rijeke Drave n...,13.12.2017. - INFORMACIJA o zahtjevu za provođ...,/doc/13122017_-_informacija_o_zahtjevu_za_prov...
4,"Radovi sanacije korita, uklanjanje nanosa i ra...",05.12.2017. - OBAVIJEST o javnoj raspravi [PDF...,/doc/05122017_-_obavijest_o_javnoj_raspravi.pdf
5,"Radovi sanacije korita, uklanjanje nanosa i ra...",05.12.2017. - STUDIJA Glavne ocjene prihvatlji...,/doc/05122017_-_studija_glavne_ocjene_prihvatl...
6,"Radovi sanacije korita, uklanjanje nanosa i ra...",07.11.2017. - INFORMACIJA o zahtjevu za provođ...,/doc/07112017_-_informacija_o_zahtjevu_za_prov...
7,Pregrada na rukavcu starog toka rijeke Drave,22.11.2017. - RJEŠENJE Ministarstva od 21. stu...,/doc/22112017_-_rjesenje_ministarstva_od_21_st...
8,Pregrada na rukavcu starog toka rijeke Drave,12.09.2017. - OBAVIJEST o javnoj raspravi [PDF...,/doc/12092017_-_obavijest_o_javnoj_raspravi.pdf
9,Pregrada na rukavcu starog toka rijeke Drave,12.09.2017. - STUDIJA Glavne ocjene prihvatlji...,/doc/12092017_-_studija_glavne_ocjene_prihvatl...


### Glavne ocjene u nadležnosti županija/grada Zagreba

In [89]:
a1 = opem[1].find_all('a')
opem_z_raw = []
for link in a1:
    red = [link.parent.parent.parent.previous_sibling.previous_sibling.get_text().strip(),
           link.get_text(),
           link.get('href')]
    opem_z_raw.append(red)
opem_z_pd = pd.DataFrame(opem_zup_raw)

In [90]:
opem_z_pd.head(10)

,0,1,2
0,Zaštita naselja Gradac i grada Pleternice od v...,18.10.2016. - OBAVIJEST o provedbi javne raspr...,/doc/18102016_-_obavijest_o_provedbi_javne_ras...
1,Ekološka revitalizacija područja starog korita...,22.09.2016. - OBJAVA javne rasprave o Studiji ...,/doc/22092016_-_objava_javne_rasprave_o_studij...
2,Održavanje korita rijeke Dretulje na području ...,19.05.2016. - OBJAVA javne rasprave o Studiji ...,/doc/19052016_-_objava_javne_rasprave_o_studij...
3,Obnova desnog nasipa rijeke Save s obaloutvrda...,16.12.2015. - OBJAVA javne rasprave o Studiji ...,/doc/16122015_-_objava_javne_rasprave_o_studij...
4,Lječilišno-turistički kompleks 'Meline' u zalj...,05.11.2015. - OBJAVA javne rasprave o Studiji ...,/doc/05112015_-_informacija_o_pocetku_javne_ra...


### Prethodne ocjene prihvatljivosti za ekološku mrežu

In [91]:
prethodne = soup.find_all('div', 'accordion')[3]

In [92]:
godpattern = re.compile(r".*?(\d{4}).*")

In [93]:
prethodne_god = prethodne.find_all('h3')
prethodne_raw = []
for god in prethodne_god:
    dokumenti = god.next_sibling.next_sibling.find_all('a')
    for dok in dokumenti:
        red = [re.sub(godpattern,
                      "\\1",
                      god.get_text().strip()),
               dok.get_text(),
               dok.get('href')]
        prethodne_raw.append(red)
prethodne_pd = pd.DataFrame(prethodne_raw)

In [94]:
prethodne_pd.head(10)

,0,1,2
0,2018,02.03.2018. - Izgradnja peradarnika za uzgoj p...,/doc/02032018_-_izgradnja_peradarnika_za_uzgoj...
1,2018,27.02.2018. - Izgradnja drvene pješačke staze ...,/doc/27022018_-_izgradnja_drvene_pjesacke_staz...
2,2018,26.02.2018. - Prenamjena poljoprivrednog zemlj...,/doc/26022018_-_prenamjena_poljoprivrednog_zem...
3,2018,21.02.2018. - Županijska razvojna strategija P...,/doc/21022018_-_zupanijska_razvojna_strategija...
4,2018,16.02.2018. - Autocesta A7 dionica između čvor...,/doc/16022018_-_autocesta_a7_dionica_izmedu_cv...
5,2018,"16.02.2018. - Uzgajalište školjaka u Blacama, ...",/doc/16022018_-_uzgajaliste_skoljaka_u_blacama...
6,2018,13.02.2018. - Rekonstrukcija (nadogradnja i pr...,/doc/13022018_-_rekonstrukcija_nadogradnja_i_p...
7,2018,09.02.2018. - III. Izmjene i dopune Prostornog...,/doc/09022018_-_iii_izmjene_i_dopune_prostorno...
8,2018,06.02.2018. - Prenamjena poljoprivrednog zemlj...,/doc/06022018_-_prenamjena_poljoprivrednog_zem...
9,2018,06.02.2018. - Transformatorska stanica KTS 10(...,/doc/06022018_-_transformatorska_stanica_kts_1...


## To do

 * Snimanje u .tsv tablice
 * Integracija u `puobot.py`
 * Twitter upload